# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [ ]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [1]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.data_processors.processor_alpaca import AlpacaProcessor
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [7]:
alpaca = AlpacaProcessor('PKB2PY7303VGIKFLW3O8', 'qa2nftQMSEIlFN4ShWSS2BCDCD5tlR10FIGyJNhw')
df = alpaca.download_data(['AMZN','NVDA','MSFT','ORCL','GOOGL','META','TSLA'], '2024-01-01', '2025-01-29', '1Min')
df

,timestamp,close,high,low,trade_count,open,volume,vwap,tic
0,2024-01-02 09:30:00-05:00,151.993,152.0200,151.3500,11931.0,151.4600,1277036.0,151.709628,AMZN
1,2024-01-02 09:31:00-05:00,152.210,152.3800,151.8700,4823.0,152.0000,397595.0,152.103890,AMZN
2,2024-01-02 09:32:00-05:00,151.470,152.2700,151.2500,3992.0,152.2000,324435.0,151.780774,AMZN
3,2024-01-02 09:33:00-05:00,151.510,151.6900,151.0650,4079.0,151.4700,312507.0,151.352808,AMZN
4,2024-01-02 09:34:00-05:00,151.330,151.5252,151.1401,3043.0,151.5100,210853.0,151.350363,AMZN
...,...,...,...,...,...,...,...,...,...
735203,2025-01-29 15:55:00-05:00,389.700,390.3999,389.2736,4163.0,389.3100,224075.0,389.895193,TSLA
735204,2025-01-29 15:56:00-05:00,389.215,389.8700,388.9200,3289.0,389.7836,157915.0,389.224906,TSLA
735205,2025-01-29 15:57:00-05:00,388.980,389.3900,388.5500,4131.0,389.2300,216087.0,388.941591,TSLA
735206,2025-01-29 15:58:00-05:00,389.050,389.7000,388.9800,4648.0,388.9800,253050.0,389.275130,TSLA


In [ ]:

df = alpaca.add_vix(df)
df = alpaca.add_technical_indicator(df)
df = alpaca.add_turbulence(df)

Data cleaning started
align start and end dates
produce full timestamp index
Start processing tickers
ticker list complete
Start concat and rename
Data clean finished!
Started adding Indicators
Running Loop
Restore Timestamps
Finished adding Indicators


/home/mike/trading/modules/FinRL/finrl/meta/data_processors/processor_alpaca.py:319: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_price_pivot = df_price_pivot.pct_change()


,timestamp,close,high,low,trade_count,open,volume,vwap,tic,vixy,macd,boll_ub,boll_lb,rsi_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2024-01-02 09:30:00-05:00,151.993,152.0200,151.3500,11931.0,151.460,1277036.0,151.709628,AMZN,15.83,0.000000,NaN,NaN,NaN,NaN,151.993000,151.993000,0.000000
1,2024-01-02 09:30:00-05:00,139.140,139.1500,138.5100,7262.0,138.590,839153.0,138.706268,GOOGL,15.83,0.000000,NaN,NaN,NaN,NaN,139.140000,139.140000,0.000000
2,2024-01-02 09:30:00-05:00,352.780,352.9288,351.1300,4305.0,351.320,319181.0,351.712465,META,15.83,0.000000,NaN,NaN,NaN,NaN,352.780000,352.780000,0.000000
3,2024-01-02 09:30:00-05:00,375.490,375.5250,373.6700,9527.0,373.690,659395.0,373.939900,MSFT,15.83,0.000000,NaN,NaN,NaN,NaN,375.490000,375.490000,0.000000
4,2024-01-02 09:30:00-05:00,491.710,492.9500,491.1878,12150.0,492.660,520255.0,492.423344,NVDA,15.83,0.000000,NaN,NaN,NaN,NaN,491.710000,491.710000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735203,2025-01-29 15:59:00-05:00,676.630,677.4100,676.0100,5700.0,677.360,218076.0,676.730920,META,42.07,-0.521151,680.531631,673.112119,47.923482,18.066022,677.760760,678.135993,24.699668
735204,2025-01-29 15:59:00-05:00,441.770,442.4300,441.6300,8198.0,442.340,357087.0,441.999651,MSFT,42.07,-0.479580,443.983836,442.055514,35.451529,48.628487,443.483450,444.079887,24.699668
735205,2025-01-29 15:59:00-05:00,123.810,123.8100,123.3900,36007.0,123.395,4885205.0,123.580105,NVDA,42.07,0.263784,123.731082,121.388628,62.742562,47.761548,122.623157,122.387638,24.699668
735206,2025-01-29 15:59:00-05:00,162.040,162.2500,162.0000,1938.0,162.230,150729.0,162.091442,ORCL,42.07,-0.042130,162.593289,162.056021,47.388306,14.228320,162.438653,162.273425,24.699668


## Data for the chosen tickers

# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [13]:
processed = df
processed


,timestamp,close,high,low,trade_count,open,volume,vwap,tic,vixy,macd,boll_ub,boll_lb,rsi_30,dx_30,close_30_sma,close_60_sma,turbulence,date
0,2024-01-02 09:30:00-05:00,151.993,152.0200,151.3500,11931.0,151.460,1277036.0,151.709628,AMZN,15.83,0.000000,NaN,NaN,NaN,NaN,151.993000,151.993000,0.000000,2024-01-02 09:30:00-05:00
1,2024-01-02 09:30:00-05:00,139.140,139.1500,138.5100,7262.0,138.590,839153.0,138.706268,GOOGL,15.83,0.000000,NaN,NaN,NaN,NaN,139.140000,139.140000,0.000000,2024-01-02 09:30:00-05:00
2,2024-01-02 09:30:00-05:00,352.780,352.9288,351.1300,4305.0,351.320,319181.0,351.712465,META,15.83,0.000000,NaN,NaN,NaN,NaN,352.780000,352.780000,0.000000,2024-01-02 09:30:00-05:00
3,2024-01-02 09:30:00-05:00,375.490,375.5250,373.6700,9527.0,373.690,659395.0,373.939900,MSFT,15.83,0.000000,NaN,NaN,NaN,NaN,375.490000,375.490000,0.000000,2024-01-02 09:30:00-05:00
4,2024-01-02 09:30:00-05:00,491.710,492.9500,491.1878,12150.0,492.660,520255.0,492.423344,NVDA,15.83,0.000000,NaN,NaN,NaN,NaN,491.710000,491.710000,0.000000,2024-01-02 09:30:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735203,2025-01-29 15:59:00-05:00,676.630,677.4100,676.0100,5700.0,677.360,218076.0,676.730920,META,42.07,-0.521151,680.531631,673.112119,47.923482,18.066022,677.760760,678.135993,24.699668,2025-01-29 15:59:00-05:00
735204,2025-01-29 15:59:00-05:00,441.770,442.4300,441.6300,8198.0,442.340,357087.0,441.999651,MSFT,42.07,-0.479580,443.983836,442.055514,35.451529,48.628487,443.483450,444.079887,24.699668,2025-01-29 15:59:00-05:00
735205,2025-01-29 15:59:00-05:00,123.810,123.8100,123.3900,36007.0,123.395,4885205.0,123.580105,NVDA,42.07,0.263784,123.731082,121.388628,62.742562,47.761548,122.623157,122.387638,24.699668,2025-01-29 15:59:00-05:00
735206,2025-01-29 15:59:00-05:00,162.040,162.2500,162.0000,1938.0,162.230,150729.0,162.091442,ORCL,42.07,-0.042130,162.593289,162.056021,47.388306,14.228320,162.438653,162.273425,24.699668,2025-01-29 15:59:00-05:00


In [12]:
processed['date'] = processed['timestamp']
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

ValueError: You are trying to merge on object and datetime64[ns, America/New_York] columns for key 'date'. If you wish to proceed you should use pd.concat

In [16]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.758536,746015200.0,4.0,0.0,2.981391,2.652102,100.0,66.666667,100.0,2.758536,2.758536,39.189999,0.0
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,43.832630,6547900.0,4.0,0.0,2.981391,2.652102,100.0,66.666667,100.0,43.832630,43.832630,39.189999,0.0
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.365304,10955700.0,4.0,0.0,2.981391,2.652102,100.0,66.666667,100.0,15.365304,15.365304,39.189999,0.0
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941090,7010200.0,4.0,0.0,2.981391,2.652102,100.0,66.666667,100.0,33.941090,33.941090,39.189999,0.0
4,2009-01-02,CAT,44.910000,46.980000,44.709999,31.579340,7117200.0,4.0,0.0,2.981391,2.652102,100.0,66.666667,100.0,31.579340,31.579340,39.189999,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [17]:
train = data_split(processed, '2024-01-01','2024-12-31')
trade = data_split(processed, '2025-01-01','2025-01-29')
print(len(train))
print(len(trade))

83897
9715


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [18]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')